# Análise Descritiva - Base Kaiserhaus
Notebook organizado para **Google Colab**.

**Objetivos:** análise exploratória inicial, qualidade dos dados e estatísticas descritivas da base de pedidos do Kaiserhaus.


## 0. Setup (instalações e imports)
Execute esta célula apenas uma vez para instalar dependências no Colab.


In [ ]:
# Se estiver no Colab, descomente a linha abaixo para garantir as dependências
# !pip install -q pandas numpy matplotlib seaborn

import os, math, textwrap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Diretório de saída para gráficos e artefatos
OUT_DIR = "out"
os.makedirs(OUT_DIR, exist_ok=True)

# Configs visuais básicas (matplotlib)
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["axes.grid"] = True
plt.rcParams["font.size"] = 10

print("✅ Bibliotecas importadas com sucesso!")


## 1. Carregar dados
Coloque seu arquivo `Base_Kaiserhaus.csv` no ambiente do Colab (ou no Google Drive) e informe o caminho.


In [ ]:
# Caminho para o arquivo CSV
caminho_csv = "Base_Kaiserhaus.csv"  # coloque o nome/caminho correto

# Leitura do arquivo CSV
df = pd.read_csv(caminho_csv)

print("📊 Dimensão da base:", df.shape)
print("📋 Colunas disponíveis:", list(df.columns))
print("\n🔍 Primeiras 5 linhas:")
df.head()


## 2. Informações básicas da base
Análise inicial da estrutura e tipos de dados.


In [ ]:
print("📈 INFORMAÇÕES BÁSICAS DA BASE")
print("=" * 50)

print(f"\n📊 Dimensões:")
print(f"   • Linhas: {df.shape[0]:,}")
print(f"   • Colunas: {df.shape[1]}")

print(f"\n🔍 Tipos de dados:")
print(df.dtypes)


## 3. Análise de valores nulos
Identificação de dados faltantes na base.


In [ ]:
print("🔍 ANÁLISE DE VALORES NULOS")
print("=" * 50)

# Contagem de valores nulos por coluna
valores_nulos = df.isnull().sum()
percentual_nulos = (valores_nulos / len(df)) * 100

# Criar DataFrame com informações de nulos
info_nulos = pd.DataFrame({
    'Coluna': valores_nulos.index,
    'Valores_Nulos': valores_nulos.values,
    'Percentual': percentual_nulos.values
})

# Filtrar apenas colunas com valores nulos
info_nulos = info_nulos[info_nulos['Valores_Nulos'] > 0].sort_values('Valores_Nulos', ascending=False)

if len(info_nulos) > 0:
    print("\n⚠️  Colunas com valores nulos:")
    print(info_nulos.to_string(index=False))
else:
    print("\n✅ Nenhum valor nulo encontrado na base!")

print(f"\n📊 Resumo:")
print(f"   • Total de valores nulos: {df.isnull().sum().sum()}")
print(f"   • Percentual geral: {(df.isnull().sum().sum() / (df.shape[0] * df.shape[1])) * 100:.2f}%")


## 4. Análise de duplicatas
Verificação de registros duplicados na base.


In [ ]:
print("🔍 ANÁLISE DE DUPLICATAS")
print("=" * 50)

# Verificar duplicatas completas
duplicatas_completas = df.duplicated().sum()

# Verificar duplicatas por pedido (assumindo que order_datetime + nome_cliente identifica pedido único)
duplicatas_pedido = df.duplicated(subset=['order_datetime', 'nome_cliente']).sum()

print(f"\n📊 Duplicatas encontradas:")
print(f"   • Duplicatas completas: {duplicatas_completas:,}")
print(f"   • Duplicatas de pedido: {duplicatas_pedido:,}")

if duplicatas_completas > 0:
    print(f"\n⚠️  Exemplo de duplicatas completas:")
    print(df[df.duplicated()].head())

if duplicatas_pedido > 0:
    print(f"\n⚠️  Exemplo de duplicatas de pedido:")
    print(df[df.duplicated(subset=['order_datetime', 'nome_cliente'])].head())


## 5. Análise de variáveis categóricas
Exploração das variáveis categóricas e suas distribuições.


In [ ]:
print("📊 ANÁLISE DE VARIÁVEIS CATEGÓRICAS")
print("=" * 50)

# Identificar colunas categóricas
colunas_categoricas = df.select_dtypes(include=['object']).columns.tolist()

print(f"\n🔍 Colunas categóricas identificadas: {colunas_categoricas}")

for coluna in colunas_categoricas:
    print(f"\n📋 {coluna.upper()}:")
    print(f"   • Valores únicos: {df[coluna].nunique()}")
    print(f"   • Valores mais frequentes:")
    
    # Mostrar top 5 valores mais frequentes
    top_valores = df[coluna].value_counts().head(5)
    for valor, count in top_valores.items():
        percentual = (count / len(df)) * 100
        print(f"     - {valor}: {count:,} ({percentual:.1f}%)")
    
    # Se há muitos valores únicos, mostrar resumo
    if df[coluna].nunique() > 10:
        print(f"   • ... e mais {df[coluna].nunique() - 5} valores únicos")


## 6. Análise de variáveis numéricas
Exploração das variáveis numéricas e identificação de outliers.


In [ ]:
print("📊 ANÁLISE DE VARIÁVEIS NUMÉRICAS")
print("=" * 50)

# Identificar colunas numéricas
colunas_numericas = df.select_dtypes(include=[np.number]).columns.tolist()

print(f"\n🔍 Colunas numéricas identificadas: {colunas_numericas}")

# Estatísticas descritivas
print(f"\n📈 ESTATÍSTICAS DESCRITIVAS:")
desc_stats = df[colunas_numericas].describe()
print(desc_stats.round(2))


## 7. Identificação de outliers
Detecção de valores extremos usando método IQR.


In [ ]:
print("🔍 IDENTIFICAÇÃO DE OUTLIERS (Método IQR)")
print("=" * 50)

def detectar_outliers_iqr(df, coluna):
    """
    Detecta outliers usando o método IQR
    """
    Q1 = df[coluna].quantile(0.25)
    Q3 = df[coluna].quantile(0.75)
    IQR = Q3 - Q1
    
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    
    outliers = df[(df[coluna] < limite_inferior) | (df[coluna] > limite_superior)]
    
    return {
        'Q1': Q1,
        'Q3': Q3,
        'IQR': IQR,
        'limite_inferior': limite_inferior,
        'limite_superior': limite_superior,
        'outliers': outliers,
        'count_outliers': len(outliers),
        'percentual_outliers': (len(outliers) / len(df)) * 100
    }

print("\n📊 Análise de outliers por variável:")

for coluna in colunas_numericas:
    resultado = detectar_outliers_iqr(df, coluna)
    
    print(f"\n🔍 {coluna.upper()}:")
    print(f"   • Q1: {resultado['Q1']:.2f}")
    print(f"   • Q3: {resultado['Q3']:.2f}")
    print(f"   • IQR: {resultado['IQR']:.2f}")
    print(f"   • Limites: [{resultado['limite_inferior']:.2f}, {resultado['limite_superior']:.2f}]")
    print(f"   • Outliers: {resultado['count_outliers']:,} ({resultado['percentual_outliers']:.1f}%)")
    
    # Mostrar alguns exemplos de outliers
    if resultado['count_outliers'] > 0:
        print(f"   • Exemplos de outliers: {resultado['outliers'][coluna].head(3).tolist()}")


## 8. Análise de valores impossíveis
Identificação de valores que não fazem sentido do ponto de vista de negócio.


In [ ]:
print("⚠️  ANÁLISE DE VALORES IMPOSSÍVEIS")
print("=" * 50)

# Verificações específicas por variável
problemas_encontrados = False

# 1. Tempos negativos
tempos_negativos = df[df['actual_delivery_minutes'] < 0]
if len(tempos_negativos) > 0:
    print(f"\n❌ Tempos de entrega negativos: {len(tempos_negativos)} registros")
    print(f"   Exemplos: {tempos_negativos['actual_delivery_minutes'].head(3).tolist()}")
    problemas_encontrados = True

# 2. Distâncias negativas
distancias_negativas = df[df['distance_km'] < 0]
if len(distancias_negativas) > 0:
    print(f"\n❌ Distâncias negativas: {len(distancias_negativas)} registros")
    problemas_encontrados = True

# 3. Valores monetários negativos
valores_negativos = df[df['total_brl'] < 0]
if len(valores_negativos) > 0:
    print(f"\n❌ Valores monetários negativos: {len(valores_negativos)} registros")
    problemas_encontrados = True

# 4. Tempos de preparo negativos
preparo_negativo = df[df['tempo_preparo_minutos'] < 0]
if len(preparo_negativo) > 0:
    print(f"\n❌ Tempos de preparo negativos: {len(preparo_negativo)} registros")
    problemas_encontrados = True

# 5. Satisfação fora do range esperado (1-5)
satisfacao_invalida = df[(df['satisfacao_nivel'] < 1) | (df['satisfacao_nivel'] > 5)]
if len(satisfacao_invalida) > 0:
    print(f"\n❌ Satisfação fora do range (1-5): {len(satisfacao_invalida)} registros")
    problemas_encontrados = True

if not problemas_encontrados:
    print("\n✅ Nenhum valor impossível encontrado!")
else:
    print(f"\n📊 Total de registros com problemas: {len(tempos_negativos) + len(distancias_negativas) + len(valores_negativos) + len(preparo_negativo) + len(satisfacao_invalida)}")


## 9. Resumo da qualidade dos dados
Síntese final da qualidade da base de dados.


In [ ]:
print("📋 RESUMO DA QUALIDADE DOS DADOS")
print("=" * 50)

# Calcular métricas de qualidade
total_registros = len(df)
total_valores_nulos = df.isnull().sum().sum()
total_duplicatas = df.duplicated().sum()

# Contar outliers totais
total_outliers = 0
for coluna in colunas_numericas:
    resultado = detectar_outliers_iqr(df, coluna)
    total_outliers += resultado['count_outliers']

# Contar valores impossíveis
valores_impossiveis = (
    len(df[df['actual_delivery_minutes'] < 0]) +
    len(df[df['distance_km'] < 0]) +
    len(df[df['total_brl'] < 0]) +
    len(df[df['tempo_preparo_minutos'] < 0]) +
    len(df[(df['satisfacao_nivel'] < 1) | (df['satisfacao_nivel'] > 5)])
)

print(f"\n📊 MÉTRICAS GERAIS:")
print(f"   • Total de registros: {total_registros:,}")
print(f"   • Total de colunas: {df.shape[1]}")
print(f"   • Uso de memória: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print(f"\n🔍 PROBLEMAS IDENTIFICADOS:")
print(f"   • Valores nulos: {total_valores_nulos:,} ({(total_valores_nulos / (total_registros * df.shape[1])) * 100:.2f}%)")
print(f"   • Duplicatas completas: {total_duplicatas:,} ({(total_duplicatas / total_registros) * 100:.2f}%)")
print(f"   • Valores impossíveis: {valores_impossiveis:,} ({(valores_impossiveis / total_registros) * 100:.2f}%)")
print(f"   • Outliers detectados: {total_outliers:,} ({(total_outliers / total_registros) * 100:.2f}%)")

# Calcular score de qualidade (0-100)
problemas_totais = total_valores_nulos + total_duplicatas + valores_impossiveis
score_qualidade = max(0, 100 - (problemas_totais / total_registros) * 100)

print(f"\n🎯 SCORE DE QUALIDADE: {score_qualidade:.1f}/100")

if score_qualidade >= 90:
    print("   ✅ Excelente qualidade!")
elif score_qualidade >= 80:
    print("   ⚠️  Boa qualidade, mas requer limpeza")
elif score_qualidade >= 70:
    print("   ⚠️  Qualidade moderada, limpeza necessária")
else:
    print("   ❌ Qualidade baixa, limpeza crítica necessária")

print(f"\n📝 PRÓXIMOS PASSOS:")
print(f"   1. Implementar limpeza de dados")
print(f"   2. Tratar valores nulos")
print(f"   3. Corrigir valores impossíveis")
print(f"   4. Decidir estratégia para outliers")
print(f"   5. Criar base limpa para análises")
